In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_9295/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession 
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import col, unix_timestamp, round
from pyspark.sql.functions import col, datediff, hour, dayofweek

sc = SparkContext.getOrCreate()

spark = SparkSession.builder \
    .appName("USPS_ML") \
    .config("spark.executor.memory", "15G") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

24/06/22 10:58:01 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.20 instead (on interface wlp61s0)
24/06/22 10:58:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/22 10:58:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.parquet("/home/pk/DAEN690/PKG_final.parquet")
#show 3 rows of our DataFrame
df.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+-----------+-----------+----------+-----------+------------+----------+-----------+------------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|    LAT_D|      LNG_D|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|EVENT_ID_O|YEARMONTH_O|EVENT_TYPE_O|EVENT_ID_D|YEARMONTH_D|EVENT_TYPE_D|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-----------

In [5]:
columns_to_drop = ["YEARMONTH_o","YEARMONTH_Ori","LAT_O","LNG_O","YEARMONTH_D","YEARMONTH_Ori","YEARMONTH_Des","LAT_D","LNG_D"]

df = df.drop(*columns_to_drop)
df.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-----------+-----------+-----------+-----------+----------+------------+----------+------------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|EVENT_ID_O|EVENT_TYPE_O|EVENT_ID_D|EVENT_TYPE_D|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-----------+-----------+-----------+-----------+----------+------------+----------+------------+
|85422102810|           058 |     37212|           2024-01-10|          02458|           PM|         12|              

In [6]:

from pyspark.sql.functions import col, dayofweek, month, year, udf
from pyspark.sql.types import IntegerType

df = df.withColumn("Origin_zip", col("Origin_zip").cast("int")) \
       .withColumn("destination_zip", col("destination_zip").cast("int"))


def categorize_zip_by_first_three_digits(zip_code):
    first_three_digits = int(str(zip_code)[:3])  
    if 0 <= first_three_digits <= 149:
        return 1  # Northeast
    elif 150 <= first_three_digits <= 199:
        return 2  # Southeast
    elif 200 <= first_three_digits <= 299:
        return 3  # South Central
    elif 300 <= first_three_digits <= 399:
        return 4  # North Central
    elif 400 <= first_three_digits <= 499:
        return 5  # Great Lakes
    elif 500 <= first_three_digits <= 599:
        return 6  # Midwest
    elif 600 <= first_three_digits <= 699:
        return 7  # South
    elif 700 <= first_three_digits <= 799:
        return 8  # Mountain
    elif 800 <= first_three_digits <= 899:
        return 9  # Southwest
    elif 900 <= first_three_digits <= 999:
        return 10  # West
    else:
        return -1  # Invalid ZIP code


categorize_udf = udf(categorize_zip_by_first_three_digits, IntegerType())


df = df.withColumn("Origin_zip_bucket", categorize_udf(col("Origin_zip")))
df = df.withColumn("destination_zip_bucket", categorize_udf(col("destination_zip")))

df = df.withColumn("day_of_week_Scheduled", dayofweek("ScheduledDeliveryDate"))
df = df.withColumn("month_Scheduled", month("ScheduledDeliveryDate"))
df = df.withColumn("year_Scheduled", year("ScheduledDeliveryDate"))

df = df.withColumn("day_of_week_Origin", dayofweek("Origin_date"))
df = df.withColumn("month_Origin", month("Origin_date"))
df = df.withColumn("year_Origin", year("Origin_date"))

df = df.withColumn("day_of_week_Destination", dayofweek("destination_date"))
df = df.withColumn("month_Destination", month("destination_date"))
df = df.withColumn("year_Destination", year("destination_date"))

# Write DataFrame to parquet file
df.write.mode('overwrite').parquet("/home/pk/DAEN690/PKG_ML.parquet")
print("done")
df.show(3)


done
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-----------+-----------+-----------+-----------+----------+------------+----------+------------+-----------------+----------------------+---------------------+---------------+--------------+------------------+------------+-----------+-----------------------+-----------------+----------------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|EVENT_ID_O|EVENT_TYPE_O|EVENT_ID_D|EVENT_TYPE_D|Origin_zip_bucket|destination_zip_bucket|day_of_week_Scheduled|month_Scheduled|year_Scheduled|day_of_week_Origin|month_Origin|year_Origin|day_of_week_Destination|month_Destination|year_Destination|
+-----------+

In [7]:
num_rows = df.count()
print("number of rows: ", num_rows)

number of rows:  1228246


In [8]:
df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- Origin_zip: integer (nullable = true)
 |-- ScheduledDeliveryDate: date (nullable = true)
 |-- destination_zip: integer (nullable = true)
 |-- MailClassCode: string (nullable = true)
 |-- total_scans: long (nullable = true)
 |-- Distinct_zip_scans: long (nullable = true)
 |-- Distinct_event_scans: long (nullable = true)
 |-- time_delta_minutes: integer (nullable = true)
 |-- Origin_date: date (nullable = true)
 |-- destination_date: date (nullable = true)
 |-- late: integer (nullable = true)
 |-- LAT_O_ROUND: double (nullable = true)
 |-- LNG_O_ROUND: double (nullable = true)
 |-- LAT_D_ROUND: double (nullable = true)
 |-- LNG_D_ROUND: double (nullable = true)
 |-- EVENT_ID_O: string (nullable = true)
 |-- EVENT_TYPE_O: string (nullable = true)
 |-- EVENT_ID_D: string (nullable = true)
 |-- EVENT_TYPE_D: string (nullable = true)
 |-- Origin_zip_bucket: integer (nullable = true)
 |-- desti

In [10]:
spark.stop()

In [11]:
###############################################################################################################

In [12]:
import findspark
findspark.init()

In [13]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession 


sc = SparkContext.getOrCreate()

spark = SparkSession.builder \
    .appName("USPS_ML") \
    .config("spark.executor.memory", "10G") \
    .getOrCreate()


spark.sparkContext.setLogLevel("ERROR")


In [14]:
df_temp = spark.read.parquet("/home/pk/DAEN690/PKG_ML.parquet")
df = df_temp.sample(withReplacement=False, fraction=0.1, seed=42)
df.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-----------+-----------+-----------+-----------+----------+------------+----------+------------+-----------------+----------------------+---------------------+---------------+--------------+------------------+------------+-----------+-----------------------+-----------------+----------------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|EVENT_ID_O|EVENT_TYPE_O|EVENT_ID_D|EVENT_TYPE_D|Origin_zip_bucket|destination_zip_bucket|day_of_week_Scheduled|month_Scheduled|year_Scheduled|day_of_week_Origin|month_Origin|year_Origin|day_of_week_Destination|month_Destination|year_Destination|
+-----------+-----

In [16]:
total_count = df.count()

count_ones = df.filter(col("late") == 1).count()

# Calculate the percentage
percentage_ones = (count_ones / total_count) * 100
print("Error % ")
print(percentage_ones )

Error % 
18.00705553387957


In [17]:
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import SparkSession


if 'label' in df.columns:
    df = df.drop('label')

label_indexer = StringIndexer(inputCol="late", outputCol="label")
df = label_indexer.fit(df).transform(df)

selected_features = [ "Origin_zip_bucket", "destination_zip_bucket",
                     "ServiceTypeCode", "MailClassCode", "total_scans", "Distinct_zip_scans",
                     "Distinct_event_scans", "time_delta_minutes", "LAT_O_ROUND", "LNG_O_ROUND",
                     "LAT_D_ROUND", "LNG_D_ROUND", "EVENT_ID_O", "EVENT_ID_D", "EVENT_TYPE_O",
                     "EVENT_TYPE_D", "label"]


data = df.select(selected_features)

categorical_cols = ["ServiceTypeCode",  "MailClassCode", "EVENT_ID_O", "EVENT_ID_D"]


indexers = [
    StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep").fit(data)
    for col in categorical_cols
]

# One-hot encode indexed categorical columns
encoders = [
    OneHotEncoder(inputCol=col+"_index", outputCol=col+"_onehot")
    for col in categorical_cols
]

pipeline = Pipeline(stages=indexers + encoders)
data_encoded = pipeline.fit(data).transform(data)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=[col + "_onehot" for col in categorical_cols], outputCol="features")
data_final = assembler.transform(data_encoded).select("features", "label")

# Split data into training and test sets (80% train, 20% test)
train_data, test_data = data_final.randomSplit([0.8, 0.2], seed=42)

# Define hyperparameters for RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label")
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()

# Set up 5-fold cross-validation
cv = CrossValidator(estimator=rf, estimatorParamMaps=param_grid, evaluator=MulticlassClassificationEvaluator(), numFolds=5)

# Train the model using cross-validation
cv_model = cv.fit(train_data)

# Make predictions on test data
predictions = cv_model.transform(test_data)

# Evaluate model performance
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

# Print best model hyperparameters
best_model = cv_model.bestModel
print("Best model hyperparameters:")
print("Num Trees:", best_model.getNumTrees)
print("Max Depth:", best_model.getOrDefault("maxDepth"))


Accuracy: 0.8258924907673368
Best model hyperparameters:
Num Trees: 10
Max Depth: 15


In [ ]:
# Accuracy: 0.8204373193740117
# Best model hyperparameters:
# Num Trees: 10
# Max Depth: 15